In [1]:
import json
import pandas
import time
import os
import recordlinkage
import jellyfish as jf
import csv

In [2]:
outputDir = 'scraped_json//'
field_agg = "_"

## Load the source file dict
sourceFiles = json.loads(open('config//sourceFiles.json', "r").read())

## Load the file schema
sourceSchema = json.loads(open('config//sourceSchema.json', "r").read())

## Load the id config
idConfig = json.loads(open('config//idConfig.json', "r").read())

In [ ]:
## If there are multiple files for any given dataset, then this function will combine all of those
## records into a single dict
def mergeSourceFiles (source, outputDir, sourceFiles):
    recordList = []
    for file in sourceFiles[source]:  
        file = json.loads(open(outputDir + file, "r", encoding="utf-8").read())
        for record in file:
            recordList.append(record)
    return recordList

In [ ]:
## Cycle through all of files per source and drop into a list of dicts
## Printing list names for reference sake
for key in sourceFiles.keys():
    print(key)
    vars()[key] = mergeSourceFiles(key, outputDir, sourceFiles)

In [ ]:
## Clean up dirty names
def mungeID(playerString):
    return ''.join(e for e in playerString if e.isalnum()).lower().replace("jr.", "").replace("st.", "state") 

In [ ]:
#create a universal identifier
#CHANGE - this is not going to be a list
#CFB-Ref reports it in chronological order so we will join based on position 1

def createNewID (fieldList, thisDict, fieldAgg):
    finalID= ''
    for i in thisDict:
        for idx, val in enumerate(fieldList):
            if (type(i[val]) is list):
                i[val]= mungeID(i[val][0])
                if (len(fieldList) -1 == idx):
                    finalID += str(i[val]).strip('[]').strip("''")
            elif (type(val) is not list):
                i[val] = mungeID(i[val])
                if (len(fieldList) - 1 == idx):
                    finalID += i[val]
                else:
                    finalID = i[val] + fieldAgg
        i['ID'] = finalID
        finalID=''

In [ ]:
createNewID(idConfig['sports247'], sports247, field_agg)
createNewID(idConfig['rivals'], rivals, field_agg)
createNewID(idConfig['cfbRef'], cfbRef, field_agg)
createNewID(idConfig['allConf'], allConf, field_agg)
createNewID(idConfig['allAmerican'], allAmerican, field_agg)
createNewID(idConfig['nflData'], nflData, field_agg)
createNewID(idConfig['ncaa'], ncaa, field_agg)

In [ ]:
df_sports247 = pandas.DataFrame(sports247)
df_rivals = pandas.DataFrame(rivals)
df_cfbRef = pandas.DataFrame(cfbRef)
df_allConf = pandas.DataFrame(allConf)
df_allAmerican = pandas.DataFrame(allAmerican)
df_nflData = pandas.DataFrame(nflData)
df_ncaa = pandas.DataFrame(ncaa)

In [ ]:
indexer = recordlinkage.Index()
indexer.block('school')
indexer.block('year')
candidate_links = indexer.index(df_sports247, df_rivals)

In [ ]:
c = recordlinkage.Compare()


c.exact('ID', 'ID', label='ID')

#c.string(str('playerName'), str('playerName'), method='damerau_levenshtein', threshold=5, label='playerName')
c.string('city', 'city', method='levenshtein', threshold=10, label='city')
#c.string('state', 'state', method='levenshtein', threshold=3, label='state')
#c.string('highSchool', 'highSchool', method='levenshtein', threshold=5, label='highSchool')

# The comparison vectors
features = c.compute(candidate_links, df_sports247, df_rivals)

features

In [ ]:
features.describe()

In [ ]:
count = 0
for record in sports247:
    for rival in rivals:
        if (record['city'] == rival['city']):
            count = count + 1
print(count)

In [ ]:
features.sum(axis=1).value_counts().sort_index(ascending=False)
matches = features[features.sum(axis=1) == 1]